# Tetszöleges környezetfüggetlen nyelvtan Chomsky normál formára hozása

_Sipos István_  
_2020.04.14_

Az órai jegyzetben az algoritmus leírását nem találtam, ezért az itt leírtakat felhasználva készítettem el a programot: [link](https://gyires.inf.unideb.hu/KMITT/b24/ch07s03.html#CNF)

In [1]:
from IPython.display import display, Math
import string

## Terminal, NonTerminal, Rule és Grammar osztályok

In [2]:
class NonTerminal():
    def __init__(self, id):
        self.id = id
        
    def __str__(self):
        return str(self.id)

    def __eq__(self, other):
        return isinstance(other, NonTerminal) and self.id == other.id
    
class Terminal():
    def __init__(self, id):
        self.id = id
        
    def __str__(self):
        return str(self.id)

    def __eq__(self, other):
        return isinstance(other, Terminal) and self.id == other.id
    

In [3]:
class Rule():
    def __init__(self, left, right):
        self.left = left
        self.right = right
        
    def __str__(self):
        return str(self.left) + r" \rightarrow " + (" ".join(str(symbol) for symbol in self.right) if len(self.right) > 0 else "\lambda ")
    
    def __eq__(self, other):
        return isinstance(other, Rule) and self.left == other.left and self.right == other.right

    @property
    def is_simple(self):
        """A szabály 'egyszerü', ha a jobb oldala csak nem terminálisokból, vagy egy darab terminálisból áll"""
        return all(isinstance(symbol, NonTerminal) for symbol in self.right) or (len(self.right) == 1 and isinstance(self.right[0], Terminal))

    @property
    def is_short(self):
        """A szabály 'rövid', ha a jobb oldala nem hosszabb mint két szimbólum"""
        return len(self.right) <= 2
    
    @property
    def is_unitary(self):
        """A szabály 'egységes', ha a jobb oldalán vagy egy terminális, vagy pontosan kettö nem terminális áll"""
        if (len(self.right) == 1 and isinstance(self.right[0], Terminal)): return True
        if (len(self.right) == 2 and isinstance(self.right[0], NonTerminal) and isinstance(self.right[1], NonTerminal)): return True
        
        return False
            

In [4]:
class Grammar():
    def __init__(self, non_terminals, terminals, start, rules):
        self.terminals = terminals
        self.non_terminals = non_terminals
        self.start = start
        self.rules = rules

    def add_rule(self, rule):
        self.rules.append(rule)

    def resolve(self, right):
        return {rule.left for rule in self.rules if rule.right == right}
    
    def __str__(self):
        return r"""
            \langle \{""" + ', '.join(str(non_terminal) for non_terminal in self.non_terminals) + r"""\}, 
            \{""" + ', '.join(str(terminal) for terminal in self.terminals) + r"""\}, 
            """ + str(self.start) + r""", 
            \{ """ + \
            ', '.join(str(rule) for rule in self.rules) +\
            r"""\} \rangle
        """

## Segédfüggvények

In [5]:
# Segédfüggvények a szabályok kényelmesebb definiálásához
def parse_symbol_sequence(symbols, string):
    return [symbols[symbol] for symbol in string.split()]

def parse_rule(symbols, non_terminal, string):
    return Rule(NonTerminal(non_terminal), parse_symbol_sequence(symbols, string))


## Implementáció

```python
CFG2CNF(grammar)
```

In [6]:
def introduce_non_terminal(non_terminals):
    candidates = [c for c in string.ascii_uppercase if c not in non_terminals]
    if len(candidates) == 0:
        raise RuntimeError("További nem terminálisok bevezetése nem lehetséges!")

    return candidates[0]

def replace_terminals_with_non_terminals(grammar):
    non_terminals = {**grammar.non_terminals}
    new_rules = []
    
    def replace_terminal(terminal):
        """
        Ellenörizzük, hogy van-e _ -> [terminal] alakú szabályunk, ha nincs vezessünk be egyet
        
        Visszatérünk a szabály bal oldalával
        """
        solves = [rule for rule in (grammar.rules + new_rules) if rule.right == [terminal]]
        if len(solves) > 0:
            return solves[0].left
        else:
            non_terminal = introduce_non_terminal(non_terminals)
            non_terminals[non_terminal] = NonTerminal(non_terminal)
            new_rule = Rule(NonTerminal(non_terminal), [terminal])
            new_rules.append(new_rule)
            
            return NonTerminal(non_terminal)

    def create_simple_rule(rule):
        """
        A paraméter nem 'egyszerü' szabályból 'egyszerü' szabályt készít, szükség esetén
        további szabályok bevezetésével
        """
        right = []
        for symbol in rule.right:
            if isinstance(symbol, NonTerminal):
                right.append(symbol)
            else:
                right.append(replace_terminal(symbol))
        return Rule(rule.left, right)
        
    def process_rules():
        for rule in grammar.rules:
            if rule.is_simple:
                yield rule
            else:
                yield create_simple_rule(rule)
        for rule in new_rules:
            yield rule
                
    rules = list(process_rules())
    
    return Grammar(non_terminals, grammar.terminals, grammar.start, rules)

def replace_long_rules(grammar):
    non_terminals = {**grammar.non_terminals}
    new_rules = []

    def shorten_rule(rule):
        short_right = rule.right[-2:]
        solves = [rule for rule in (grammar.rules + new_rules) if rule.right == short_right]
        if len(solves) > 0:
            left = solves[0].left
        else:
            non_terminal = introduce_non_terminal(non_terminals)
            left = NonTerminal(non_terminal)
            non_terminals[non_terminal] = NonTerminal(non_terminal)
            new_rule = Rule(left, short_right)
            new_rules.append(new_rule)
        
        remainder = Rule(rule.left, [*rule.right[:-2], left])
        
        if remainder not in (grammar.rules + new_rules):
            if remainder.is_short:
                new_rules.append(remainder)
            else:
                shorten_rule(remainder)

    def process_rules():
        for rule in grammar.rules:
            if rule.is_short:
                yield rule
            else:
                shorten_rule(rule)

        for rule in new_rules:
            yield rule

    rules = list(process_rules())
    
    return Grammar(non_terminals, grammar.terminals, grammar.start, rules)

def deriveable(non_terminal, other, rules, seen = None):
    if seen == None:
        seen = []
    
    directs = [rule.right[0] for rule in rules \
               if rule.left == other \
               and len(rule.right) == 1 \
               and rule.right[0] not in seen]

    if len(directs) == 0:
        return False
    
    if non_terminal in directs:
        return True
    
    for indirect in directs:
        if deriveable(non_terminal, indirect, rules, seen + directs):
            return True
    
    return False

def extend(grammar):
    """
    Megadjuk az eredeti nyelvtannal ekvivalens G' Chomsky-féle normálalakú nyelvtant.
    Ehhez két lépésre van szükség. Első lépésben meghatározunk egy U(Z) halmazt
    minden olyan Z nemterminálishoz, mely levezethető legalább egy másik
    nemterminálisból a G 2 nyelvtanban és szerepel olyan H 2 halmazban lévő szabály
    bal oldalán, amelynek jobb oldalán egy terminális vagy pedig két nemterminális
    betű áll. Az U(Z) halmaz tartalmazni fogja az összes olyan nemterminálist,
    melyből egy vagy több lépésben levezethető a Z betű.
    """
    U = {}
    for non_terminal in grammar.non_terminals.values():
        U[non_terminal.id] = set()
        
        for other in grammar.non_terminals.values():
            if (deriveable(non_terminal, other, grammar.rules)):
                U[non_terminal.id].add(other.id)
    
    """
    Második lépésben a H' szabályhalmazba átvesszük a H 2 szabályhalmaz mindazon
    szabályait, melyek jobb oldalán egy terminális betű vagy pedig kettő nemterminális
    található, majd hozzávesszük mindazon szabályokat, melyeket úgy kapunk, hogy a már
    átvett szabályok bal oldalán szereplő betűt a hozzá tartozó U halmaz elemeivel
    helyettesítjük.
    """
    new_rules = []
    
    for non_terminal in U:
        for rule in grammar.rules:
            if rule.left.id == non_terminal:
                for substitute in U[non_terminal]:
                    new_rule = Rule(NonTerminal(substitute), rule.right[:])
                    if new_rule not in (grammar.rules + new_rules):
                        new_rules.append(new_rule)
                        
    return Grammar(grammar.non_terminals, grammar.terminals, grammar.start, grammar.rules + new_rules)

def filter(grammar):
    """
    A nem szabályos (A -> BC vagy A -> a alakú) szabályok elhagyása
    """
    return Grammar(grammar.non_terminals, grammar.terminals, grammar.start, \
                   [rule for rule in grammar.rules if rule.is_unitary] \
                   )


def CFG2CNF(grammar):
    """
    A paraméter grammar környezetfüggetlen nyelvtant nyelvtan CNF-ra hozza!
    """
    
    print('Az eredeti nyelvtan:')
    display(Math(str(grammar)))
    
    print('Terminálisok elkülönítése:')
    g1 = replace_terminals_with_non_terminals(grammar)
    display(Math(str(g1)))
    
    print('A hosszú szabályok lerövidítése:')
    g2 = replace_long_rules(g1)
    display(Math(str(g2)))
    
    print('Kiterjesztés:')
    g3 = extend(g2)
    display(Math(str(g3)))

    print('Elhagyás:')
    g4 = filter(g3)
    display(Math(str(g4)))

    return g4

## Példák

### Példa 1

In [7]:
# Terminálisok
terminals = {n:Terminal(n) for n in ['a', 'b', 'c']}

# Nem terminálisok
non_terminals = {n:NonTerminal(n) for n in ['A', 'B', 'S']}

# Az összes szimbólum egy halmazban
symbols = {**terminals, **non_terminals}

g = Grammar(non_terminals, terminals, NonTerminal('S'), [
    parse_rule(symbols, 'S', 'A'),
    parse_rule(symbols, 'S', 'B c'),
    parse_rule(symbols, 'A', 'c A B'),
    parse_rule(symbols, 'A', 'a'),
    parse_rule(symbols, 'B', 'b A'),
    parse_rule(symbols, 'B', 'B b S a'),
    ])

cnf = CFG2CNF(g)


Az eredeti nyelvtan:


<IPython.core.display.Math object>

Terminálisok elkülönítése:


<IPython.core.display.Math object>

A hosszú szabályok lerövidítése:


<IPython.core.display.Math object>

Kiterjesztés:


<IPython.core.display.Math object>

Elhagyás:


<IPython.core.display.Math object>

### Példa 2

In [8]:
# Terminálisok
terminals = {n:Terminal(n) for n in ['a', 'b', 'c']}

# Nem terminálisok
non_terminals = {n:NonTerminal(n) for n in ['A', 'B', 'S']}

# Az összes szimbólum egy halmazban
symbols = {**terminals, **non_terminals}

g = Grammar(non_terminals, terminals, NonTerminal('S'), [
    parse_rule(symbols, 'S', 'A B a b a'),
    parse_rule(symbols, 'A', 'B'),
    parse_rule(symbols, 'A', 'c'),
    parse_rule(symbols, 'B', 'A b A'),
    parse_rule(symbols, 'B', 'S'),
    ])

cnf = CFG2CNF(g)

Az eredeti nyelvtan:


<IPython.core.display.Math object>

Terminálisok elkülönítése:


<IPython.core.display.Math object>

A hosszú szabályok lerövidítése:


<IPython.core.display.Math object>

Kiterjesztés:


<IPython.core.display.Math object>

Elhagyás:


<IPython.core.display.Math object>

### Példa 3

In [9]:
# Terminálisok
terminals = {n:Terminal(n) for n in ['a', 'b', 'c']}

# Nem terminálisok
non_terminals = {n:NonTerminal(n) for n in ['D', 'S']}

# Az összes szimbólum egy halmazban
symbols = {**terminals, **non_terminals}

g = Grammar(non_terminals, terminals, NonTerminal('S'), [
    parse_rule(symbols, 'S', 'a S c'),
    parse_rule(symbols, 'S', 'D'),
    parse_rule(symbols, 'D', 'b D'),
    parse_rule(symbols, 'D', 'b'),
    ])

cnf = CFG2CNF(g)

Az eredeti nyelvtan:


<IPython.core.display.Math object>

Terminálisok elkülönítése:


<IPython.core.display.Math object>

A hosszú szabályok lerövidítése:


<IPython.core.display.Math object>

Kiterjesztés:


<IPython.core.display.Math object>

Elhagyás:


<IPython.core.display.Math object>